- 1	Muscle loss
- 2	2-Oxoglutarate
- 3	3-Aminoisobutyrate
- 4	Adipate
- 5	Betaine
- 6	Creatinine
- 7	Hypoxanthine
- 8	N,N-Dimethylglycine




- 1	3
- 1	4
- 1	5
- 1	7
- 2	7
- 6	7
- 8	5


Directed GRAPH of the example - Comparison Accuracy 0.7532

In [1]:
from kdm.models import KDMSequentialJointClassModel
import kdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np
from keras import metrics
from keras import losses
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Dense
import keras
from pandas import read_csv, DataFrame
import os
os.environ["KERAS_BACKEND"] = "tensorflow"


In [2]:
df = read_csv('cachexia.csv')

In [3]:
df.head()

,Patient ID,Muscle loss,"1,6-Anhydro-beta-D-glucose",1-Methylnicotinamide,2-Aminobutyrate,2-Hydroxyisobutyrate,2-Oxoglutarate,3-Aminoisobutyrate,3-Hydroxybutyrate,3-Hydroxyisovalerate,...,Tryptophan,Tyrosine,Uracil,Valine,Xylose,cis-Aconitate,myo-Inositol,trans-Aconitate,pi-Methylhistidine,tau-Methylhistidine
0,PIF_178,cachexic,40.85,65.37,18.73,26.05,71.52,1480.30,56.83,10.07,...,259.82,290.03,111.05,86.49,72.24,237.46,135.64,51.94,157.59,160.77
1,PIF_087,cachexic,62.18,340.36,24.29,41.68,67.36,116.75,43.82,79.84,...,83.10,167.34,46.99,109.95,192.48,333.62,376.15,217.02,307.97,130.32
2,PIF_090,cachexic,270.43,64.72,12.18,65.37,23.81,14.30,5.64,23.34,...,82.27,60.34,31.50,59.15,2164.62,330.30,86.49,58.56,145.47,83.93
3,NETL_005_V1,cachexic,154.47,52.98,172.43,74.44,1199.91,555.57,175.91,25.03,...,235.10,323.76,30.57,102.51,125.21,1863.11,247.15,75.94,249.64,254.68
4,PIF_115,cachexic,22.20,73.70,15.64,83.93,33.12,29.67,76.71,69.41,...,103.54,142.59,44.26,160.77,186.79,101.49,749.95,98.49,84.77,79.84


In [4]:
# finish just Muscle loss, 3-Aminoisobutyrate, Adipate, Betaine and Hypoxanthine

filtered_df = df[['Muscle loss', '3-Aminoisobutyrate', 'Adipate', 'Betaine', 'Hypoxanthine']]

In [5]:
filtered_df.head()

,Muscle loss,3-Aminoisobutyrate,Adipate,Betaine,Hypoxanthine
0,cachexic,1480.30,38.09,109.95,97.51
1,cachexic,116.75,327.01,244.69,82.27
2,cachexic,14.30,131.63,116.75,114.43
3,cachexic,555.57,144.03,278.66,223.63
4,cachexic,29.67,15.03,391.51,66.69


In [6]:
# transform musche loss in 0-1
filtered_df['Muscle loss'] = filtered_df['Muscle loss'].apply(lambda x: 1 if x == 'cachexic' else 0)

/var/folders/2s/yq898nms72b5qxqbwz_hx8j80000gn/T/ipykernel_73525/903024324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Muscle loss'] = filtered_df['Muscle loss'].apply(lambda x: 1 if x == 'cachexic' else 0)


In [7]:
filtered_df.head()

,Muscle loss,3-Aminoisobutyrate,Adipate,Betaine,Hypoxanthine
0,1,1480.30,38.09,109.95,97.51
1,1,116.75,327.01,244.69,82.27
2,1,14.30,131.63,116.75,114.43
3,1,555.57,144.03,278.66,223.63
4,1,29.67,15.03,391.51,66.69


## Prediction Power

In [8]:

encoded_size = 4
dim_y = 2
encoder = keras.layers.Identity()
n_comp = 77
sequences = [
    {
        'type': 'merge'
    },
    ]
kdm_model = KDMSequentialJointClassModel(encoded_size=encoded_size,
                                         dim_y=dim_y,
                                         encoder=encoder,
                                         n_comp=n_comp,
                                         sequences=sequences,
                                         sigma=0.5)

In [9]:
kdm_model.compile(optimizer=optimizers.Adam(learning_rate=5e-5),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [12]:
data = []
for row in filtered_df.iterrows():
    y = int(row[1]['Muscle loss'])
    x = row[1][1:].values
    data.append((x, y))
    

In [23]:
input = np.array([x for x, y in data])
y_out = np.array([y for x, y in data])
output = np.array([[0,1] if y == 1 else [1,0] for x, y in data])

In [20]:
output

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1,

In [21]:
kdm_model.init_components(np.array(input), np.array(output),
                                init_sigma=True, sigma_mult=0.5, super_index=0, index=0)

In [24]:
check_1 = kdm_model.evaluate(np.array(input), np.array(y_out))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4964 - sparse_categorical_accuracy: 0.8037  


In [25]:

print('Full Dataset loss:', check_1[0])
print('Full Dataset accuracy:', check_1[1])

Full Dataset loss: 0.5318797826766968
Full Dataset accuracy: 0.7792207598686218


## Training Power

In [103]:
combined_y = np.array([[y, [0,1]] if y == 1 else [y, [1,0]] for x, y in data])

/var/folders/2s/yq898nms72b5qxqbwz_hx8j80000gn/T/ipykernel_73525/3391731745.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combined_y = np.array([[y, [0,1]] if y == 1 else [y, [1,0]] for x, y in data])


In [104]:
X_train, X_test, y_train, y_test = train_test_split(
    input, combined_y, test_size=0.1, random_state=42)

In [105]:
len(X_train)

69

In [126]:

encoded_size = 4
dim_y = 2
encoder = keras.layers.Identity()
n_comp = 69
sequences = [
    {
        'type': 'merge'
    },
    ]
kdm_model = KDMSequentialJointClassModel(encoded_size=encoded_size,
                                         dim_y=dim_y,
                                         encoder=encoder,
                                         n_comp=n_comp,
                                         sequences=sequences,
                                         sigma=0.5)

kdm_model.compile(optimizer=optimizers.Adam(learning_rate=5e-7),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [127]:
y_train

array([[0, list([1, 0])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [0, list([1, 0])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [0, list([1, 0])],
       [1, list([0, 1])],
       [1, list([0, 1])],
       [0, l

In [128]:
real_out = np.array([y[1] for y in y_train])

kdm_model.init_components(np.array(X_train), np.array(real_out),
                                init_sigma=True, sigma_mult=0.5, super_index=0, index=0)

In [129]:
check_full = kdm_model.evaluate(np.array(input), np.array(y_out))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4766 - sparse_categorical_accuracy: 0.7841  


In [130]:
print('Full Dataset loss:', check_full[0])
print('Full Dataset accuracy:', check_full[1])

Full Dataset loss: 0.5080514550209045
Full Dataset accuracy: 0.7792207598686218


In [131]:
check_test = kdm_model.evaluate(np.array(X_test), np.array([y[0] for y in y_test]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6675 - sparse_categorical_accuracy: 0.5000


In [132]:
print('Test loss:', check_test[0])
print('Test accuracy:', check_test[1])

Test loss: 0.6674742698669434
Test accuracy: 0.5


In [133]:
history = kdm_model.fit(
    np.array(X_train),  # Your training data
    np.array(y_out),  # Your training labels
    batch_size=2,
    epochs=100, 
    verbose=1,  # Detailed logging
    validation_split=0.1,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

Epoch 1/100


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7910 - sparse_categorical_accuracy: 0.4282 - val_loss: 0.9639 - val_sparse_categorical_accuracy: 0.5714
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - loss: 0.8959 - sparse_categorical_accuracy: 0.4450 - val_loss: 0.9645 - val_sparse_categorical_accuracy: 0.5714
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step - loss: 0.8723 - sparse_categorical_accuracy: 0.5460 - val_loss: 0.9650 - val_sparse_categorical_accuracy: 0.5714
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 1.0666 - sparse_categorical_accuracy: 0.5410 - val_loss: 0.9656 - val_sparse_categorical_accuracy: 0.5714
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: 1.2113 - sparse_categorical_accuracy: 0.5678 - val_loss: 0.9661 - val_sparse_categorical_accuracy: 0.5714
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 0.8881 - sparse_categorical_accuracy: 0.6009 - val_loss: 0.9668 - val_sparse_categorical_accuracy: 0.5714
Epoch 7/100


In [134]:

check_full = kdm_model.evaluate(np.array(input), np.array(y_out))
check_test = kdm_model.evaluate(np.array(X_test), np.array([y[0] for y in y_test]))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4680 - sparse_categorical_accuracy: 0.8180 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6800 - sparse_categorical_accuracy: 0.6250


In [135]:

print('Full Dataset loss:', check_full[0])
print('Full Dataset accuracy:', check_full[1])


print('Test loss:', check_test[0])
print('Test accuracy:', check_test[1])


Full Dataset loss: 0.5098720192909241
Full Dataset accuracy: 0.7922077775001526
Test loss: 0.6799858808517456
Test accuracy: 0.625
